<a href="https://colab.research.google.com/github/sh-Gabriel/Atrial-Enlargement-Detection/blob/main/TCCJhonsons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports

from google.colab import drive

import matplotlib.pyplot as plt
# from google.colab.patches import cv2_imshow

from PIL import Image
import numpy  as np
import pandas as pd
import cv2    as cv
np.random.seed(42)
import random
import os

import glob
import csv
from itertools import islice
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import tensorflow as tf
from tensorflow import keras

from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.pipeline import Pipeline
from google.colab.patches import cv2_imshow
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from sklearn import metrics
from tensorflow import keras
from keras import layers 

In [5]:
!ls sample_data

anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [6]:
#mount and defines

drive.mount('/content/drive',force_remount=True)
PATH             = 'drive/MyDrive/TCC/Dataset_V1/'
norm             = "N"
doente           = "AAE_EP"
left             = "Torax_LAT_E"
right            = "Torax_LAT_D"
stop_n = stop_s  = 6
#shape_w, shape_h = 2928, 2328
shape_w, shape_h = 224,224
EPOCHS           = 12
# !git clone https://github.com/pjreddie/darknet.git
# !cd drive/MyDrive/TCC/darknet; make
!nvidia-smi --query-gpu=memory.total --format=csv
!nvidia-smi

Mounted at /content/drive
memory.total [MiB]
15109 MiB
Thu Oct 27 17:01:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                 

Coloquei os defs e a chamada do join_open_files tudo junto, porque basicamente eles fazem uma coisa só que é o join_open_files e assim, não precisa ficar rodando os defs de novo toda vez que muda alguma coisa

In [7]:
from google.colab.patches import cv2_imshow

#1 chamada = 1 load em 1 classe com 1 lateralidade
# retorno do vetor com os nomes das imagens
def openImgs(classe,lateralidade):
  r_v = [] 
  for subdir, dirs, files in os.walk(PATH+classe):
    if (files and lateralidade in subdir):
      r_v += [subdir+'/'+files[0]]
  return shuffle(r_v, random_state = 42)

#Retorna vetores para entrada de modelo
#  X valores e Y labels
#  Y label da imagem (1 = presença de doença)

def join(lateralidade):
  #print("Accessing files...")
  X=[]
  Y=[]
  for f in islice(openImgs(norm, lateralidade), 0, stop_n):
      X += [f]
      Y += [0]

  for f in islice(openImgs(doente, lateralidade), 0, stop_s):
      X += [f]
      Y += [1]
  return X,Y

def join_open_files(lateralidade=left, reshape=True):
  X, y = join(lateralidade)
  Xl = []
  for i in X:
    # cv2_imshow(cv.imread(i, 0))
    if reshape:
      Xl.append(cv.resize(cv.imread(i), (shape_w, shape_h)))
    else:
      Xl.append(cv.imread(i))
  return np.array(Xl), np.array(y)

X, y = join_open_files() #Coloca as imagens já abertas nos vetores de 'treino e teste'

In [ ]:
def custom_VGG16():
  vgg16 = keras.applications.VGG16(input_shape=(shape_w, shape_h, 3), include_top=False, weights="imagenet");

  last_layer = vgg16.get_layer('block5_pool')
  last_output = last_layer.output
  extraLayer = tf.keras.layers.GlobalMaxPooling2D()(last_output)
  extraLayer = tf.keras.layers.Dense(512, activation='relu')(extraLayer)
  extraLayer = tf.keras.layers.Dropout(0.5)(extraLayer)
  extraLayer = tf.keras.layers.Dense(2, activation='sigmoid')(extraLayer)
  
  model = keras.Model(vgg16.input, extraLayer)
  
  return model
    
    
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [ ]:
def custom_InceptionV3():
  iv3 = keras.applications.InceptionV3(input_shape=(shape_w, shape_h, 3), include_top=False, weights="imagenet");

  last_layer  = iv3.get_layer('mixed10')
  last_output = last_layer.output
  extraLayer = tf.keras.layers.GlobalMaxPooling2D()(last_output)
  extraLayer = tf.keras.layers.Dense(512, activation='relu')(extraLayer)
  extraLayer = tf.keras.layers.Dropout(0.5)(extraLayer)
  extraLayer = tf.keras.layers.Dense(2, activation='sigmoid')(extraLayer)

  model = keras.Model(iv3.input, extraLayer)

  return model
    
    

In [ ]:
def custom_ResNet50V2():
  resnet50v2 = keras.applications.ResNet50V2(input_shape=(shape_w, shape_h, 3), include_top=False, weights="imagenet");

  return resnet50v2

#A principio tá rodando sem precisar colocar as layers no final

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state = 0)
# clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
# clf.score(X_test, y_test)
from sklearn import svm
#Leave one patient out using SVC from above
import numpy as np
from keras.callbacks import EarlyStopping,ModelCheckpoint
from sklearn.model_selection import LeaveOneOut

def LOO2(X, y, model):
  # fold_var = 1
  loo = LeaveOneOut()
  loo.get_n_splits(X)
  results = []
  nsamples, nx, ny, channels = X.shape

  checkpoint_path = "drive/MyDrive/TCC/checkpoints/cp.ckpt"
  checkpoint_dir = os.path.dirname(checkpoint_path)

  cp = "drive/MyDrive/TCC/checkpoints/"
 
  model.save_weights('drive/MyDrive/TCC/checkpoints/cp.ckpt')
  #loss resolvei o (None, ) (None, x, y,)
  model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
  fname ="weights.{epoch:02d}-{val_loss:.2f}.hdf5"
  
  for train_index, test_index in loo.split(X):
    checkpoint = ModelCheckpoint('drive/MyDrive/TCC/checkpoints/teste.cptk', monitor='val_accuracy', verbose=1, mode='max')
    callback_list = [checkpoint]

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # print(X_train.shape)
    # print(y_train.shape)
    # print(X_test.shape)
    # print(y_test.shape)
    history = model.fit(X_train, y_train, epochs=EPOCHS, callbacks=checkpoint)
    # transfer_model.fit(train1_data, train1_labels, batch_size = 15, epochs=5, validation_data=(test1_data, test1_labels) callbacks=[lr_reduce,checkpoint])

    model.load_weights(checkpoint_path)
    # model.load_weights(os.path.dirname(cp+"weights.{epoch:02d}-{val_loss:.2f}.hdf5" ))
    results += [model.evaluate(X_test, y_test)[1]]
    # print(results)
    # results = dict(zip(model.metrics_names, results))
    tf.keras.backend.clear_session()
    # fold_var += 1
  return results

In [ ]:
print(np.sum(LOO2(X, y, custom_VGG16()))/12)

Epoch 1/12
1/1 [==============================] - ETA: 0s - loss: 25.5023 - accuracy: 0.5455
Epoch 1: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


1/1 [==============================] - 5s 5s/step - loss: 25.5023 - accuracy: 0.5455
Epoch 2/12
1/1 [==============================] - ETA: 0s - loss: 64.3898 - accuracy: 0.4545
Epoch 2: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


1/1 [==============================] - 3s 3s/step - loss: 64.3898 - accuracy: 0.4545
Epoch 3/12
1/1 [==============================] - ETA: 0s - loss: 89.7905 - accuracy: 0.5455
Epoch 3: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


1/1 [==============================] - 4s 4s/step - loss: 89.7905 - accuracy: 0.5455
Epoch 4/12
1/1 [==============================] - ETA: 0s - loss: 294.6555 - accuracy: 0.4545
Epoch 4: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


1/1 [==============================] - 3s 3s/step - loss: 294.6555 - accuracy: 0.4545
Epoch 5/12
1/1 [==============================] - ETA: 0s - loss: 13.3618 - accuracy: 0.4545
Epoch 5: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


...

1/1 [==============================] - 3s 3s/step - loss: 6.3792 - accuracy: 0.5455
Epoch 9/12
1/1 [==============================] - ETA: 0s - loss: 1.4066 - accuracy: 0.5455
Epoch 9: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


1/1 [==============================] - 3s 3s/step - loss: 1.4066 - accuracy: 0.5455
Epoch 10/12
1/1 [==============================] - ETA: 0s - loss: 0.9918 - accuracy: 0.4545
Epoch 10: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


1/1 [==============================] - 3s 3s/step - loss: 0.9918 - accuracy: 0.4545
Epoch 11/12
1/1 [==============================] - ETA: 0s - loss: 0.7789 - accuracy: 0.4545
Epoch 11: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


1/1 [==============================] - 4s 4s/step - loss: 0.7789 - accuracy: 0.4545
Epoch 12/12
1/1 [==============================] - ETA: 0s - loss: 0.5161 - accuracy: 0.7273
Epoch 12: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


1/1 [==============================] - 0s 92ms/step - loss: 0.0000e+00 - accuracy: 1.0000
0.5


In [ ]:
print(np.sum(LOO2(X, y, custom_InceptionV3()))/12)
#custom_InceptionV3().summary()

Epoch 1/12
1/1 [==============================] - ETA: 0s - loss: 1.7026 - accuracy: 0.5455
Epoch 1: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


1/1 [==============================] - 43s 43s/step - loss: 1.7026 - accuracy: 0.5455
Epoch 2/12
1/1 [==============================] - ETA: 0s - loss: 16.6641 - accuracy: 0.5455
Epoch 2: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


1/1 [==============================] - 36s 36s/step - loss: 16.6641 - accuracy: 0.5455
Epoch 3/12
1/1 [==============================] - ETA: 0s - loss: 3.8442 - accuracy: 0.5455
Epoch 3: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


1/1 [==============================] - 37s 37s/step - loss: 3.8442 - accuracy: 0.5455
Epoch 4/12
1/1 [==============================] - ETA: 0s - loss: 0.1365 - accuracy: 0.9091
Epoch 4: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


1/1 [==============================] - 34s 34s/step - loss: 0.1365 - accuracy: 0.9091
Epoch 5/12
1/1 [==============================] - ETA: 0s - loss: 2.9847 - accuracy: 0.6364
Epoch 5: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


...

1/1 [==============================] - 33s 33s/step - loss: 0.3091 - accuracy: 0.9091
Epoch 9/12
1/1 [==============================] - ETA: 0s - loss: 0.8037 - accuracy: 0.9091
Epoch 9: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


1/1 [==============================] - 35s 35s/step - loss: 0.8037 - accuracy: 0.9091
Epoch 10/12
1/1 [==============================] - ETA: 0s - loss: 0.0038 - accuracy: 1.0000
Epoch 10: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


1/1 [==============================] - 34s 34s/step - loss: 0.0038 - accuracy: 1.0000
Epoch 11/12
1/1 [==============================] - ETA: 0s - loss: 1.4074 - accuracy: 0.6364
Epoch 11: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


1/1 [==============================] - 34s 34s/step - loss: 1.4074 - accuracy: 0.6364
Epoch 12/12
1/1 [==============================] - ETA: 0s - loss: 8.1487e-04 - accuracy: 1.0000
Epoch 12: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


1/1 [==============================] - 0s 466ms/step - loss: 7.1860 - accuracy: 0.0000e+00
0.3333333333333333


In [ ]:
print(np.sum(LOO2(X, y, custom_ResNet50V2()))/12)

Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                         

1/1 [==============================] - 40s 40s/step - loss: 11.6983 - accuracy: 0.3636
Epoch 2/12
1/1 [==============================] - ETA: 0s - loss: 14.2775 - accuracy: 0.4545
Epoch 2: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


1/1 [==============================] - 23s 23s/step - loss: 14.2775 - accuracy: 0.4545
Epoch 3/12
1/1 [==============================] - ETA: 0s - loss: 0.7590 - accuracy: 0.9091
Epoch 3: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


1/1 [==============================] - 26s 26s/step - loss: 0.7590 - accuracy: 0.9091
Epoch 4/12
1/1 [==============================] - ETA: 0s - loss: 5.1443 - accuracy: 0.7273
Epoch 4: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


1/1 [==============================] - 23s 23s/step - loss: 5.1443 - accuracy: 0.7273
Epoch 5/12
1/1 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


...

1/1 [==============================] - 22s 22s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/12
1/1 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


1/1 [==============================] - 21s 21s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/12
1/1 [==============================] - ETA: 0s - loss: 9.7201e-04 - accuracy: 1.0000
Epoch 10: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


1/1 [==============================] - 22s 22s/step - loss: 9.7201e-04 - accuracy: 1.0000
Epoch 11/12
1/1 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 11: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


1/1 [==============================] - 21s 21s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 12/12
1/1 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 12: saving model to drive/MyDrive/TCC/checkpoints/teste.cptk


1/1 [==============================] - 0s 307ms/step - loss: 655.5429 - accuracy: 0.0000e+00
0.5


Verificar pra fazer algo customizado e alterar o classiicador por meio de um laço, fica bala

Antes tava fazendo 50% treino e 50% teste, por isso o 0.333333...

> Mount e Defines

>  Funções que abrem a imagem

> *   openImgs()
*   join()
*   joi_open_file()
*   def VGG16(), InceptionV3() e ResNet50V2()
*   chamadas das redes usando o Leave-One-Out
*   chamada dos summary() das redes

> Problemas não resolvidos
>* por algum caralho de motivo, a rede recebe os parametros de shape_w e shape_h invertidos. Se tenta inverter a rede dá um erro de "ResourceExhaustedError: Graph execution error"

* por outro caralho de motivo, conforme vai rodando, vão sendo criados mais checkpoints que filho de pobre esperando bolsa familia 

> Problemas resolvidos

In [ ]:
custom_VGG16().summary()
custom_ResNet50V2().summary()
custom_InceptionV3().summary()

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.333
session = InteractiveSession(config=config)

# model code as usual down here